# Bostrom Revenue and Price Model

In [1]:
from src.data_extractors import get_pools, get_prices
from config import BOSTROM_LCD_CLIENT

BASE_INVESTMINTING_AMOUNT_AMPERE = 100_000_000
BASE_INVESTMINTING_AMOUNT_VOLT = 1_000_000_000
BASE_INVESTMINTING_PERIOD_AMPERE = 30
BASE_INVESTMINTING_PERIOD_VOLT = 30

VALIDATOR_COMMISSION = 0.05
NUMBER_DEGEN_ITERATIONS = 3

## Optimal State
PRICE_HYDROGEN_OPTIMAL = 0.2
REVENUE_HYDROGEN_OPTIMAL = 0.4
BOOT_REVENUE_OPTIMAL = 0.2  # stake + restake

## Actual State

In [2]:
staked_boot =  BOSTROM_LCD_CLIENT.staking.pool().bonded_tokens.amount
supply_boot = [int(_coin['amount']) for _coin in BOSTROM_LCD_CLIENT.bank.total()[0].to_data() if _coin['denom'] == 'boot'][0]
inflation = float(BOSTROM_LCD_CLIENT.mint.inflation())
community_tax = float(BOSTROM_LCD_CLIENT.distribution.parameters()['community_tax'])

revenue_boot = supply_boot / staked_boot * inflation * (1 - VALIDATOR_COMMISSION) * (1 - community_tax)
revenue_with_restake_boot = (1 + revenue_boot/365) ** 365 - 1

print(f'BOOT Supply {supply_boot:>22,}\n'
      f'BOOT Staked {staked_boot:>22,}\n'
      f'BOOT Inflation  {inflation:.2%}\n'
      f'BOOT Inflation Community Tax  {community_tax:.0%}\n'
      f'BOOT Staking Revenue without restake  {revenue_boot:.2%}\n'
      f'BOOT Staking Revenue with restake     {revenue_with_restake_boot:.2%}\n')

price_df = get_prices(get_pools(networks=['bostrom', 'osmosis']), zero_fee=True)

price_ampere_actual = price_df.loc['hydrogen', 'milliampere'] * 1_000
price_volt_actual = price_df.loc['hydrogen', 'millivolt'] * 1_000
price_hydrogen_actual = price_df.loc['boot', 'hydrogen']

print(f'Ampere price {price_ampere_actual:>12,.1f} H\n'
      f'Volt price   {price_volt_actual:>12,.1f} H\n'
      f'H price      {price_hydrogen_actual:>14,.3f} BOOT\n')
print(f"OSMO price"
      f"\n\t- in bostrom {price_df.loc['hydrogen', 'uosmo in bostrom'] * price_df.loc['boot', 'hydrogen'] * 1_000_000:>,.0f} BOOT"
      f"\n\t- in osmosis {price_df.loc['boot in osmosis', 'uosmo'] * 1_000_000:>,.0f} BOOT")
print(f"ATOM price"
      f"\n\t- in bostrom {price_df.loc['hydrogen', 'uatom in bostrom'] * price_df.loc['boot', 'hydrogen'] * 1_000_000:>,.0f} BOOT"
      f"\n\t- in osmosis {price_df.loc['boot in osmosis', 'uatom in osmosis'] * 1_000_000:>,.0f} BOOT")

BOOT Supply  1,143,264,876,290,015
BOOT Staked    281,041,932,038,145
BOOT Inflation  5.46%
BOOT Inflation Community Tax  10%
BOOT Staking Revenue without restake  18.99%
BOOT Staking Revenue with restake     20.91%

Ampere price  3,721,359.9 H
Volt price   35,842,107.4 H
H price               0.215 BOOT

OSMO price
	- in bostrom 29,795,807 BOOT
	- in osmosis 29,758,426 BOOT
ATOM price
	- in bostrom 442,472,515 BOOT
	- in osmosis 442,281,972 BOOT


## Revenue Strategies
### Degen
Stake BOOT with restake -> Buy BOOT for HYDROGEN -> Stake BOOT with restake - 3 times

In [3]:
degen_revenue_actual = degen_revenue_optimal = 0
staked_boot_actual = staked_boot_optimal = 1

for _ in range(NUMBER_DEGEN_ITERATIONS):
    degen_revenue_actual +=  staked_boot_actual * revenue_with_restake_boot
    staked_boot_actual *= price_hydrogen_actual

    degen_revenue_optimal +=  staked_boot_optimal * BOOT_REVENUE_OPTIMAL
    staked_boot_optimal *= PRICE_HYDROGEN_OPTIMAL

print(f'Degen revenue actual {degen_revenue_actual:7.1%}\n'
      f'Degen revenue optimal {degen_revenue_optimal:6.1%}')

Degen revenue actual   26.4%
Degen revenue optimal  24.8%


### Staking + Investminting

Stake BOOT with retake -> Investmint HYDROGEN

In [4]:
hydrogen_revenue_actual = \
    (1 * 365 / (BASE_INVESTMINTING_AMOUNT_VOLT * BASE_INVESTMINTING_PERIOD_VOLT) * price_volt_actual +
     1 * 365 / (BASE_INVESTMINTING_AMOUNT_AMPERE * BASE_INVESTMINTING_PERIOD_AMPERE) * price_ampere_actual) / 2
staking_investminting_revenue_actual = revenue_with_restake_boot + price_hydrogen_actual * hydrogen_revenue_actual
staking_investminting_revenue_optimal = BOOT_REVENUE_OPTIMAL + PRICE_HYDROGEN_OPTIMAL * REVENUE_HYDROGEN_OPTIMAL

print(f'Staking + Investminting revenue actual {staking_investminting_revenue_actual:7.1%}\n'
      f'Staking + Investminting revenue optimal {staking_investminting_revenue_optimal:6.1%}')

Staking + Investminting revenue actual   30.5%
Staking + Investminting revenue optimal  28.0%


### Investminting
Buy HYDROGEN for BOOT -> Investmint HYDROGEN

In [5]:
print(f'Investminting revenue actual {hydrogen_revenue_actual:7.1%}\n'
      f'Investminting revenue optimal {REVENUE_HYDROGEN_OPTIMAL:6.1%}')

Investminting revenue actual   44.4%
Investminting revenue optimal  40.0%


## Price Model

In [6]:
price_ampere_optimal = BASE_INVESTMINTING_AMOUNT_AMPERE * BASE_INVESTMINTING_PERIOD_AMPERE / 365 * REVENUE_HYDROGEN_OPTIMAL
price_volt_optimal = BASE_INVESTMINTING_AMOUNT_VOLT * BASE_INVESTMINTING_PERIOD_VOLT / 365 * REVENUE_HYDROGEN_OPTIMAL

print(f'AMPERE price actual {price_ampere_actual:>,.1f} optimal {price_ampere_optimal:>,.1f} can decrease by {price_ampere_actual/price_ampere_optimal:.2f} times')
print(f'VOLT price actual {price_volt_actual:>,.1f} optimal {price_volt_optimal:>,.1f} can decrease by {price_volt_actual/price_volt_optimal:.2f} times')
print(f'H price actual {price_hydrogen_actual:.3f} optimal {PRICE_HYDROGEN_OPTIMAL} can decrease by {price_hydrogen_actual/PRICE_HYDROGEN_OPTIMAL:.2f} times')

AMPERE price actual 3,721,359.9 optimal 3,287,671.2 can decrease by 1.13 times
VOLT price actual 35,842,107.4 optimal 32,876,712.3 can decrease by 1.09 times
H price actual 0.215 optimal 0.2 can decrease by 1.08 times
